In [1]:
import pandas as pd
from google.cloud import bigquery
from google.cloud.exceptions import Conflict
import os

In [2]:
df_gastos_2025 = pd.read_csv('../_data/GPCP21748622989979-012025-122025.csv',sep=";")
df_gastos_2024 = pd.read_csv('../_data/GPCP21738336106217-012024-122024.csv',sep=";")
df_gastos_2023 = pd.read_csv('../_data/GPCP21706283634418-012023-122023.csv',sep=";")
df_gastos_2022 = pd.read_csv('../_data/GPCP21674495098299-012022-122022.csv',sep=";")

/var/folders/v1/x67btc_n7qzdc9sww5ttffnc018gtf/T/ipykernel_65353/2474531291.py:2: DtypeWarning: Columns (20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gastos_2024 = pd.read_csv('../_data/GPCP21738336106217-012024-122024.csv',sep=";")
/var/folders/v1/x67btc_n7qzdc9sww5ttffnc018gtf/T/ipykernel_65353/2474531291.py:3: DtypeWarning: Columns (20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gastos_2023 = pd.read_csv('../_data/GPCP21706283634418-012023-122023.csv',sep=";")


In [11]:
df_gastos = pd.concat([df_gastos_2022, df_gastos_2023, df_gastos_2024, df_gastos_2025], ignore_index=True)

In [31]:
df_gastos


,EJERCICIO,TIPO_INSTITUCION,BD_TIPO_INSTITUCION,INSTITUCION,BD_INSTITUCION,UNIDAD_PRES,BD_UNIDAD_PRES,LINEA_TRABAJO,BD_LINEA_TRABAJO,AREA_GESTION,...,RUBRO,DESCRIPCION_DE_RUBRO,CUENTA,DESCRIPCION_DE_CUENTA,MES,PROGRAMADO,MODIFICACION,PROGRAMADO_MODIFICADO,DEVENGADO,COMPROMISO
0,2022,1,Gobierno Central,100,0100 Organo Legislativo ...,1,01 Dirección y Administración Institucional...,101,0101 Dirección Superior 2022,1,...,51,51 Remuneraciones ...,517,517 Indemnizaciones ...,1,0.0,48723.68,48723.68,48723.68,48723.68
1,2022,1,Gobierno Central,100,0100 Organo Legislativo ...,1,01 Dirección y Administración Institucional...,101,0101 Dirección Superior 2022,1,...,51,51 Remuneraciones ...,515,515 Contribuciones patronales a inst de seg so...,1,5387.0,88248.55,93635.55,93815.48,93635.55
2,2022,1,Gobierno Central,100,0100 Organo Legislativo ...,1,01 Dirección y Administración Institucional...,101,0101 Dirección Superior 2022,1,...,54,54 Adquisiciones de bienes y servicios ...,543,543 Servicios generales y arrendamientos ...,1,6195.0,-6195.0,0.0,0.0,0.0
3,2022,1,Gobierno Central,100,0100 Organo Legislativo ...,1,01 Dirección y Administración Institucional...,101,0101 Dirección Superior 2022,1,...,54,54 Adquisiciones de bienes y servicios ...,544,544 Pasajes y viáticos ...,1,15000.0,-15000.0,0.0,0.0,0.0
4,2022,1,Gobierno Central,100,0100 Organo Legislativo ...,1,01 Dirección y Administración Institucional...,101,0101 Dirección Superior 2022,1,...,54,54 Adquisiciones de bienes y servicios ...,541,541 Bienes de uso y consumo ...,1,36686.0,-36415.25,270.75,270.75,270.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277207,2025,4,Empresa Publica,4301,4301 Administración Nacional de Acueductos y A...,7,07 Inversión 2025,703,"0703 Desarrollo de la Infraestructura, Admini...",7,...,54,54 Adquisiciones de bienes y servicios ...,545,"545 Consultorías, estudios e investigaciones ...",11,35840.0,-31415.22,4424.78,0.0,4424.78
277208,2025,4,Empresa Publica,4301,4301 Administración Nacional de Acueductos y A...,7,07 Inversión 2025,703,"0703 Desarrollo de la Infraestructura, Admini...",7,...,55,55 Gastos financieros y otros ...,555,"555 Impuestos, tasas y derechos ...",12,69.0,0.0,69.0,0.0,0.0
277209,2025,4,Empresa Publica,4301,4301 Administración Nacional de Acueductos y A...,7,07 Inversión 2025,703,"0703 Desarrollo de la Infraestructura, Admini...",7,...,54,54 Adquisiciones de bienes y servicios ...,541,541 Bienes de uso y consumo ...,12,645.0,0.0,645.0,0.0,0.0
277210,2025,4,Empresa Publica,4301,4301 Administración Nacional de Acueductos y A...,7,07 Inversión 2025,703,"0703 Desarrollo de la Infraestructura, Admini...",7,...,54,54 Adquisiciones de bienes y servicios ...,549,549 Crédito fiscal ...,12,4744.0,-2359.61,2384.39,0.0,2300.89


In [33]:
df_gastos.columns

Index(['EJERCICIO', 'TIPO_INSTITUCION', 'BD_TIPO_INSTITUCION', 'INSTITUCION',
       'BD_INSTITUCION', 'UNIDAD_PRES', 'BD_UNIDAD_PRES', 'LINEA_TRABAJO',
       'BD_LINEA_TRABAJO', 'AREA_GESTION', 'BD_AREA_GESTION', 'FUENTE_RECURS',
       'BD_FTE_RECURSOS', 'FUENTE_FINANC', 'BD_FTE_FINANCIAMIENTO', 'RUBRO',
       'DESCRIPCION_DE_RUBRO', 'CUENTA', 'DESCRIPCION_DE_CUENTA', 'MES',
       'PROGRAMADO', 'MODIFICACION', 'PROGRAMADO_MODIFICADO', 'DEVENGADO',
       'COMPROMISO'],
      dtype='object')

In [35]:
# List of columns to convert
cols_to_convert = [
    'PROGRAMADO',
    'MODIFICACION',
    'PROGRAMADO_MODIFICADO',
    'DEVENGADO',
    'COMPROMISO'
]

# Apply pd.to_numeric to the specified columns
df_gastos[cols_to_convert] = df_gastos[cols_to_convert].apply(pd.to_numeric, errors='coerce')


In [29]:
def df_to_bq(dataframe: pd.DataFrame, dataset_id: str, table_name: str):
    """
    Uploads a pandas DataFrame to a new BigQuery table.

    The BigQuery schema is automatically inferred from the DataFrame dtypes.
    This function will fail if the destination table already exists and contains data.

    Args:
        dataframe (pd.DataFrame): The pandas DataFrame to upload.
        dataset_id (str): The ID of the BigQuery dataset (e.g., 'my_dataset').
        table_name (str): The name for the new table to be created (e.g., 'my_new_table').
    """
    # --- 1. Initialize the BigQuery Client ---
    # This assumes you have authenticated correctly (e.g., by setting the
    # GOOGLE_APPLICATION_CREDENTIALS environment variable).
    try:
        client = bigquery.Client()
        print("BigQuery client initialized successfully.")
    except Exception as e:
        print(f"Error initializing BigQuery client: {e}")
        return

    # --- 2. Construct the full table ID ---
    project_id = client.project
    table_id = f"{project_id}.{dataset_id}.{table_name}"
    print(f"Target table ID: {table_id}")

    # --- 3. Configure the Load Job ---
    # The schema is inferred from the DataFrame dtypes.
    # We use WRITE_EMPTY to ensure the job fails if the table already exists and has data,
    # effectively enforcing the "create new table" behavior.
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_EMPTY,
    )

    # --- 4. Execute the load job ---
    try:
        print(f"Starting job to load DataFrame into table '{table_name}'...")
        # The client library handles schema detection from the DataFrame automatically.
        load_job = client.load_table_from_dataframe(
            dataframe, table_id, job_config=job_config
        )

        # Waits for the job to complete.
        load_job.result()
        print("Load job finished.")

        # --- 5. Verify the result ---
        destination_table = client.get_table(table_id)
        print(f"Successfully loaded {destination_table.num_rows} rows into table '{table_id}'.")

    except Conflict as e:
        # This specific exception occurs if the table is not empty (due to WRITE_EMPTY).
        print(f"Error: Table '{table_id}' already exists and is not empty. {e}")
    except Exception as e:
        print(f"An unexpected error occurred during the load job: {e}")

In [8]:
dataset_id = "goes_data"
table_id = "gastos_presupuesto"

In [37]:
df_to_bq(
            dataframe=df_gastos,
            dataset_id=dataset_id,
            table_name=table_id,
        )

BigQuery client initialized successfully.
Target table ID: speca-sandbox-argolis.goes_data.gastos_presupuesto
Starting job to load DataFrame into table 'gastos_presupuesto'...


/Users/speca/Dev/goes_agents_poc/.venv/lib/python3.12/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Load job finished.
Successfully loaded 277212 rows into table 'speca-sandbox-argolis.goes_data.gastos_presupuesto'.
